In [1]:
import numpy as np

class LipschitzCalculator:
    def __init__(self, network_layers):
        """
        Initialize calculator with network architecture
        network_layers: list of weight matrices for each layer
        """
        self.layers = network_layers
        
    def calculate_matrix_norm(self, matrix, norm_type='spectral'):
        """
        Calculate matrix norm (default: spectral norm)
        """
        if norm_type == 'spectral':
            # Spectral norm is the largest singular value
            return np.linalg.norm(matrix, ord=2)
        elif norm_type == 'frobenius':
            # Frobenius norm
            return np.linalg.norm(matrix, ord='fro')
        
    def calculate_product_bound(self):
        """
        Calculate Lipschitz constant using product of matrix norms
        This provides an upper bound on the true Lipschitz constant
        """
        lipschitz_constant = 1.0
        
        for layer in self.layers:
            layer_norm = self.calculate_matrix_norm(layer)
            lipschitz_constant *= layer_norm
            
        return lipschitz_constant
    
    def calculate_power_iteration(self, num_iterations=100):
        """
        Calculate Lipschitz constant using power iteration method
        This provides a more accurate estimate of the true Lipschitz constant
        """
        # Start with random vector
        current_vector = np.random.randn(self.layers[0].shape[1])
        current_vector = current_vector / np.linalg.norm(current_vector)
        
        for _ in range(num_iterations):
            # Forward pass
            intermediate = current_vector.copy()
            for layer in self.layers:
                intermediate = layer @ intermediate
                
            # Backward pass
            for layer in reversed(self.layers):
                intermediate = layer.T @ intermediate
                
            # Normalize
            norm = np.linalg.norm(intermediate)
            current_vector = intermediate / norm
            
        # Final forward pass to get the Lipschitz constant
        intermediate = current_vector.copy()
        for layer in self.layers:
            intermediate = layer @ intermediate
            
        return np.sqrt(np.sum(intermediate ** 2))

# Example usage
def demonstrate_lipschitz_calculation():
    # Create a simple network with 2 layers
    layer1 = np.array([[0.5, 0.3],
                       [0.2, 0.8]])
    layer2 = np.array([[0.7, 0.4],
                       [0.1, 0.6]])
    
    network = [layer1, layer2]
    calculator = LipschitzCalculator(network)
    
    # Calculate using different methods
    product_bound = calculator.calculate_product_bound()
    power_iteration = calculator.calculate_power_iteration()
    
    print(f"Lipschitz constant upper bound (product method): {product_bound:.4f}")
    print(f"Lipschitz constant estimate (power iteration): {power_iteration:.4f}")
    
    # Demonstrate effect of scaling weights
    scaled_network = [2 * layer1, 2 * layer2]
    scaled_calculator = LipschitzCalculator(scaled_network)
    
    scaled_product_bound = scaled_calculator.calculate_product_bound()
    scaled_power_iteration = scaled_calculator.calculate_power_iteration()
    
    print(f"\nAfter scaling weights by 2:")
    print(f"Lipschitz constant upper bound: {scaled_product_bound:.4f}")
    print(f"Lipschitz constant estimate: {scaled_power_iteration:.4f}")

if __name__ == "__main__":
    demonstrate_lipschitz_calculation()

Lipschitz constant upper bound (product method): 0.8699
Lipschitz constant estimate (power iteration): 0.8556

After scaling weights by 2:
Lipschitz constant upper bound: 3.4796
Lipschitz constant estimate: 3.4223
